### Turing Machines

In [17]:
%run core.ipynb

In [51]:
from array import array


class Tape:

    def __init__(self, values):
        self.values = array("I", values)
        self.pos = 0
        
    def __repr__(self):
        return "".join([f"{v}" for v in self.values[:self.pos]] + 
                        [_blu(self.values[self.pos])] + 
                        [f"{v}" for v in self.values[self.pos + 1:]])
    
    def __getitem__(self, i):
        return self.values[i]
    
    def __iter__(self):
        return iter(self.values)
    
    def __len__(self):
        return len(self.values)
    
    def read(self):
        return self.values[self.pos]

    def write(self, v):
        self.values[self.pos] = v

    def move(self, mv):
        if mv == "L":
            if self.pos == 0:
                self.values.insert(0, 0)
            else:
                self.pos -= 1
        else:
            if self.pos == len(self.values) - 1:
                self.values.append(0)
            self.pos += 1


In [52]:
class TuringMachine:
    
    def __init__(self, instructions):
        self.instructions = instructions
        
    def __getitem__(self, i):
        return self.instructions[i]

    def __call__(self, tape, max_iterations=None):
        i, instr, mv = 0, 0, None
        while not mv == "S":
            if max_iterations is not None and i > max_iterations:
                raise StopIteration(f"max iterations {max_iterations} reached")
            value = tape.read()
            next_instr, new_value, mv = self.instructions[instr][value]
            yield instr, next_instr, value, new_value, mv
            tape.write(new_value)
            tape.move(mv)
            instr = next_instr
            i += 1


In [53]:
def run_turing_machine(tm, tape, max_iterations=None):
    for instr, _, value, new_value, move in tm(tape, max_iterations=max_iterations):
        print(f"{instr}: {value} -> {new_value} mv={move}:", tape)
    print()
    print(f"result: {tape}")

In [54]:
UN_PLUS_ONE = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 1, "R"),
    },
    1: {
        0: (0, 1, "S"), 
        1: (1, 1, "R"),
    },
}
un_plus_one = TuringMachine(UN_PLUS_ONE)

In [55]:
tape = Tape([0, 0, 1, 1, 1, 0, 0])
run_turing_machine(un_plus_one, tape)

0: 0 -> 0 mv=R: 0011100
0: 0 -> 0 mv=R: 0011100
0: 1 -> 1 mv=R: 0011100
1: 1 -> 1 mv=R: 0011100
1: 1 -> 1 mv=R: 0011100
1: 0 -> 1 mv=S: 0011100

result: 0011110


In [56]:
tape = Tape([0, 1, 1, 1, 1, 1, 0, 0])
run_turing_machine(un_plus_one, tape)

0: 0 -> 0 mv=R: 01111100
0: 1 -> 1 mv=R: 01111100
1: 1 -> 1 mv=R: 01111100
1: 1 -> 1 mv=R: 01111100
1: 1 -> 1 mv=R: 01111100
1: 1 -> 1 mv=R: 01111100
1: 0 -> 1 mv=S: 01111100

result: 01111110


In [57]:
UN_TIMES_TWO = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 0, "R"),
    },
    1: {
        0: (2, 1, "L"), 
        1: (1, 1, "R"),
    },
    2: {
        0: (3, 0, "R"), 
        1: (4, 0, "R"),
    },
    3: {
        0: (0, 1, "S"), 
        1: (3, 1, "R"),
    },
    4: {
        0: (5, 1, "L"), 
        1: (4, 1, "R"),
    },
    5: {
        0: (2, 1, "L"), 
        1: (5, 1, "L"),
    },
}
un_times_two = TuringMachine(UN_TIMES_TWO)

In [58]:
tape = Tape([0, 1])
run_turing_machine(un_times_two, tape)

0: 0 -> 0 mv=R: 01
0: 1 -> 0 mv=R: 01
1: 0 -> 1 mv=L: 000
2: 0 -> 0 mv=R: 001
3: 1 -> 1 mv=R: 001
3: 0 -> 1 mv=S: 0010

result: 00110


In [59]:
tape = Tape([0, 1, 1, 1, 0])
tape = Tape([1, 1, 1])
run_turing_machine(un_times_two, tape)

0: 1 -> 0 mv=R: 111
1: 1 -> 1 mv=R: 011
1: 1 -> 1 mv=R: 011
1: 0 -> 1 mv=L: 0110
2: 1 -> 0 mv=R: 0111
4: 1 -> 1 mv=R: 0101
4: 0 -> 1 mv=L: 01010
5: 1 -> 1 mv=L: 01011
5: 0 -> 1 mv=L: 01011
2: 1 -> 0 mv=R: 01111
4: 1 -> 1 mv=R: 00111
4: 1 -> 1 mv=R: 00111
4: 1 -> 1 mv=R: 00111
4: 0 -> 1 mv=L: 001110
5: 1 -> 1 mv=L: 001111
5: 1 -> 1 mv=L: 001111
5: 1 -> 1 mv=L: 001111
5: 0 -> 1 mv=L: 001111
2: 0 -> 0 mv=R: 011111
3: 1 -> 1 mv=R: 011111
3: 1 -> 1 mv=R: 011111
3: 1 -> 1 mv=R: 011111
3: 1 -> 1 mv=R: 011111
3: 1 -> 1 mv=R: 011111
3: 0 -> 1 mv=S: 0111110

result: 01111110


In [60]:
def _cvrt(X):
    if isinstance(X, int):
        return [X]
    elif isinstance(X, str):
        return str2bin(X)
    elif isinstance(X, (list, tuple)):
        if len(X) == 1:
            return X[0]
    return X


def int2bin(x):
    return [int(i) for i in bin(x)[2:]]


def bin2int(X):
    return sum(2**i if x == 1 else 0 for i, x in enumerate(reversed(_cvrt(X))))


def int2xnbin(X):
    y = []
    for x in _cvrt(X):
        for n in int2bin(x):
            if n == 0:
                y.append(0)
            elif n == 1:
                y.append(1)
                y.append(0)
        y.append(1)
        y.append(1)
        y.append(0)
    return _cvrt(y)


def xnbin2int(y):
    i, x, X = 0, [], []
    while i < len(y):
        if y[i] == 0:
            x.append(0)
            i += 1
        elif y[i] == 1:
            i += 1
            if y[i] == 0:
                x.append(1)
                i += 1
            else:
                X.append(bin2int(x))
                i += 2
                x = []
    return _cvrt(X)


In [61]:
x = 167
int2bin(x)
assert x == bin2int(int2bin(x))

int2xnbin(x)
assert x == xnbin2int(int2xnbin(x))

X = [6, 8]
int2xnbin(X)
assert X == xnbin2int(int2xnbin(X))


[1, 0, 1, 0, 0, 1, 1, 1]

[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]

[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]

In [62]:
XN_TIMES_TWO = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 0, "R"),
    },
    1: {
        0: (0, 1, "R"), 
        1: (2, 0, "R"),
    },
    2: {
        0: (3, 1, "R"), 
    },
    3: {
        0: (0, 1, "S"),
    },
}
xn_times_two = TuringMachine(XN_TIMES_TWO)

In [63]:
x = 167
tape = Tape(int2xnbin(x))
run_turing_machine(xn_times_two, tape, max_iterations=50)
xnbin2int(tape)

0: 1 -> 0 mv=R: 1001000101010110
1: 0 -> 1 mv=R: 0001000101010110
0: 0 -> 0 mv=R: 0101000101010110
0: 1 -> 0 mv=R: 0101000101010110
1: 0 -> 1 mv=R: 0100000101010110
0: 0 -> 0 mv=R: 0100100101010110
0: 0 -> 0 mv=R: 0100100101010110
0: 1 -> 0 mv=R: 0100100101010110
1: 0 -> 1 mv=R: 0100100001010110
0: 1 -> 0 mv=R: 0100100011010110
1: 0 -> 1 mv=R: 0100100010010110
0: 1 -> 0 mv=R: 0100100010110110
1: 0 -> 1 mv=R: 0100100010100110
0: 1 -> 0 mv=R: 0100100010101110
1: 1 -> 0 mv=R: 0100100010101010
2: 0 -> 1 mv=R: 0100100010101000
3: 0 -> 1 mv=S: 01001000101010010

result: 010010001010100110


334

#### Universal Turing Machines

In [64]:
s = "10101101101001011010100111010010110101111010000111010010101110100010111010100011010010110110101010101101010101101010100"
bin2int(s)
assert "".join(str(k) for k in int2bin(bin2int(s))) == s

450813704461563958982113775643437908

In [99]:
def generate_instructions(k):
    
    def iter_instructions(k):
        i, K = 0, int2bin(k)
        values, mv = [], None
        yield (0, 0, "R")
        while i < len(K):
            if K[i] == 0:
                values.append(0)
                i += 1
            elif K[i:i + 1] == [1, 0]:
                values.append(1)
                i += 2
            elif K[i:i + 2] == [1, 1, 0]:
                mv = "R"
                i += 3
            elif K[i:i + 3] == [1, 1, 1, 0]:
                mv = "L"
                i += 4
            elif K[i:i + 4] == [1, 1, 1, 1, 0]:
                mv = "S"
                i += 5
            else:
                raise Exception("should not happen")
            if mv is not None:
                if not values:
                    valus = [0, 0]
                elif values == [1]:
                    valus = [0, 1]
                yield (bin2int(values[:-1]), values[-1], mv)
                values, mv = [], None
                
    print(k, int2bin(k))
    I = {}
    for i, instr in enumerate(iter_instructions(k)):
        if i % 2 == 0:
            I[i] = {0: instr}
        else:
            I[i][1] = instr
    return I


class UniversalTuringMachine(TuringMachine):
    
    def __init__(self, k):
        super().__init__(generate_instructions(k))
        self.k = k
        
    def __str__(self):
        return f"T_{self.k}"


In [100]:
n = 1
n = 3
n = 5
tm = UniversalTuringMachine(n)
print(tm)
pprint(tm.instructions)


5 [1, 0, 1]


Exception: should not happen

In [89]:
n = 12
n = 1
for k in range(n):
    tm = UniversalTuringMachine(k)
    print(tm)
    pprint(tm.instructions)
#     tape = Tape([0, 1, 1, 0, 0])
#     run_turing_machine(tm, tape, max_iterations=15)

0 [0]
T_0
{0: {0: (0, 0, 'R')}}
